In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 31 17:19:37 2020

@author: lixiangk
"""

from interface import *

# Generate an input file for `Genesis 1.3` version 4

In [5]:
workdir = r'temp'
os.chdir(workdir)

P0 = 17 # MeV/c
gamma0 = M2G(P0)

lambda0 = 100e-6
lam_u = 30e-3
delz = lam_u/2

setup = Namelist('setup', 
                 rootname = 'test',
                 lattice = 'gen4lat.lat',
                 beamline = 'tunnel',
                 gamma0 = gamma0,
                 lambda0 = lambda0,
                 delz = delz,
                 seed = 1,
                 npart = 8192,
                 nbins = 4,
                 one4one = False,
                 shotnoise = True)

time = Namelist('time',
                s0 = 0,
                slen = 15e-3,
                sample = 1,
                time = True)

profile_gauss = Namelist('profile_gauss', 
                         label = 'beampro',
                         c0 = 150.0,
                         s0 = 2.5e-3,
                         sig = 1e-3)

profile_step = Namelist('profile_step', 
                         label = 'beampro',
                         c0 = 150.0,
                         s_start = 0e-3,
                         s_end = 5e-3)

beam = Namelist('beam',
                gamma = gamma0,
                delgam = gamma0*0.5e-2,
                current = '@beampro',
                ex = 4e-6,
                ey = 4e-6,
                betax = 19.99072,
                betay = 0.74887,
                alphax = 10.93,
                alphay = 3.25,
                bunch = 0,
                emod = 0)

importbeam = Namelist('importbeam',
                      file = 'test.0.par.h5',
                      time = True)

importdist = Namelist('importdistribution',
                      file = 'test_dist.h5',
                      charge = 1e-9,
                      slicewidth = 0.01)

efield = Namelist('efield',
                  rmax = 0,
                  nz = 1,
                  nphi = 2,
                  ngrid = 170)

field = Namelist('field',
                 power = 0,
                 phase = 0,
                 dgrid = 20e-3,
                 ngrid = 201)

track = Namelist('track',
                 output_step = 1,
                 field_dump_step = 0,
                 beam_dump_step = 268)

g4 = Genesis4(setup, time, profile_step, beam, importdist, field, track)
print(g4.output)

g4.output = g4.output.lower()
g4.write('test.in')
#g4.qsub('test.sh', 'test.in')

&SETUP 
 ROOTNAME=test
 LATTICE=gen4lat.lat
 BEAMLINE=tunnel
 GAMMA0=33.28319610805032
 LAMBDA0=0.0001
 DELZ=0.015
 SEED=1
 NPART=8192
 NBINS=4
 ONE4ONE=False
 SHOTNOISE=True
&END

&TIME 
 S0=0
 SLEN=0.015
 SAMPLE=1
 TIME=True
&END

&PROFILE_STEP 
 LABEL=beampro
 C0=150.0
 S_START=0.0
 S_END=0.005
&END

&BEAM 
 GAMMA=33.28319610805032
 DELGAM=0.1664159805402516
 CURRENT=@beampro
 EX=4e-06
 EY=4e-06
 BETAX=19.99072
 BETAY=0.74887
 ALPHAX=10.93
 ALPHAY=3.25
 BUNCH=0
 EMOD=0
&END

&IMPORTDISTRIBUTION 
 FILE=test_dist.h5
 CHARGE=1e-09
 SLICEWIDTH=0.01
&END

&FIELD 
 POWER=0
 PHASE=0
 DGRID=0.02
 NGRID=201
&END

&TRACK 
 OUTPUT_STEP=1
 FIELD_DUMP_STEP=0
 BEAM_DUMP_STEP=268
&END




# Generating input files for parameter scan

In [6]:
#%% Batch generating inputs for parameter scan
for i in np.arange(1, 5):
    
    # shot noise
    seed = i
    
    fname = 'modulated.%d' % (seed)
    setup.set(rootname = fname, seed = seed)
    #setup.set(nbins = 1, shotnoise = False)
    importdist.set(file = 'test_dist2.h5')
    g4 = Genesis4(setup, time, profile_step, importdist, field, track)
    
    g4.write(fname+'.in')
    g4.sbatch(fname+'.sh', fname+'.in', cmd1 = 'module add gnu openmpi phdf5/1.10.6')

# Astra format to hdf5 format used by `Genesis 1.3` version 4

In [ ]:
data = pd_loadtxt('beam_modulated.ini')
data[1:,2] += data[0,2]
data[1:,5] += data[0,5]

select = (data[:,2]>=-100e-6*40)*(data[:,2]<=100e-6*40); data = data[select]

x, y, z = data[:,0:3].T[:]
t = z/g_c

Px, Py, Pz = data[:,3:6].T[:]
xp = Px/Pz
yp = Py/Pz
p = np.sqrt(Px**2+Py**2+Pz**2)/1e6/g_mec2

import h5py

f = h5py.File('test_dist.h5', 'w')

dx = f.create_dataset("x", data = x)
dy = f.create_dataset("y", data = y)
dy = f.create_dataset("t", data = t)

dxp = f.create_dataset("xp", data = xp)
dyp = f.create_dataset("yp", data = yp)
dp = f.create_dataset("p", data = p)

f.close()

# slice-wise parameters from txt to hdf format

In [ ]:
import h5py
def slice2hdf5(inputName = None, ext = '.h5'):
    try:
        import h5py
    except Exception as err:
        print(err)
        
    if inputName is None:
        inputName = get_file('.*')
        
    file = open(inputName, 'r')
    line = file.readline()
    i = 1
    while True:
        if not line:
            break
        match = re.search(r'COLUMNS' , line.upper())
        if match:
            keys = line.split()[2:]
            break
        line = file.readline()
        i += 1
    file.close()
    
    data = np.loadtxt(inputName, skiprows = i)  
    
    path, base, _ = fileparts(inputName)
    outputName = os.path.join(path, base+ext); print(outputName)
    
    #outputName = 'test.in.h5'
    with h5py.File(outputName, 'w') as file:
        for i, key in enumerate(keys):
            dset = file.create_dataset(key, data = data[:,i])
    
    return

slice2hdf5('slice@beam_und.ini.001')        